# Regressão 01 - tarefa 03 - transformações em X e Y

Carregue os pacotes necessários e a base de gorjetas.

### I. Modelo no valor da gorjeta

1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo em ```tip```, explicada por ```sex, smoker, diner e net_bill```.  
2. Remova as variáveis não significantes.  
3. observe o gráfico de resíduos em função de ```net_bill```  
4. teste transformar ```net_bill``` no log e um polinômio. Escolha o melhor modelo.

In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


tips = sns.load_dataset("tips")
# calculando a taxa de gorjeta
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips['net_bill'] = tips['total_bill'] - tips['tip']
tips.head() 

##### 1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo em tip, explicada por sex + smoker + diner + net_bill

In [137]:

import patsy as patsy 
import statsmodels.api as sm

# coloquei todas as variáveis para ver o que é significativo pois apenas as especificadas não são suficientes, apenas net_bill acaba sendo significativa o que não geraria um modelo de regressão multivariada.
y, X = patsy.dmatrices('tip ~ sex + smoker + net_bill + size + day + time', tips)

modelo = sm.OLS(y, X).fit()
print(modelo.summary())

##### 2. Remova as variáveis não significantes.

In [138]:

y, X = patsy.dmatrices('tip ~ net_bill + size', tips)    
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

##### 3. observe o gráfico de resíduos em função de net_bill

In [139]:

tips['residues'] = modelo.resid
sns.scatterplot(x = 'net_bill', y = 'residues', data = tips)
plt.axhline(y=0, color='r', linestyle='--')

##### 4. teste transformar net_bill no log e um polinômio. Escolha o melhor modelo.

In [140]:

y, X = patsy.dmatrices('tip ~ np.log(net_bill) + size', tips)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

In [141]:
y, X = patsy.dmatrices('tip ~ np.power(net_bill, 2) + size', tips)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

O melhor modelo é o que tem a variável net_bill sem transformação e a variável size.

### II. Modelo no valor do percentual da gorjeta

1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo no log de ```tip```, explicado por ```sex, smoker, diner e net_bill```.
2. Remova as variáveis não significantes.
3. Observe o gráfico de resíduos em função de ```net_bill```
4. Teste transformar ```net_bill``` no log e um polinômio. Escolha o melhor modelo.
5. Do modelo final deste item, calcule o $R^2$ na escala de ```tip``` (sem o log). Compare com o modelo do item 1. Qual tem melhor coeficiente de determinação?

##### 1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo no log de ```tip```, explicado por ```sex, smoker, diner e net_bill```.

In [142]:
# 1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo no log de tip, explicado
y, X = patsy.dmatrices('np.log(tip) ~ sex + smoker + net_bill + time', tips)
modelo1 = sm.OLS(y, X).fit()
print(modelo1.summary())

##### 2. Remova as variáveis não significantes.


In [143]:
# 2. Remova as variáveis não significantes.
y, X = patsy.dmatrices('np.log(tip) ~ net_bill', tips)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

##### 3. Observe o gráfico de resíduos em função de ```net_bill```


In [144]:
# 3. Observe o gráfico de resíduos em função de net_bill
tips['residues'] = modelo.resid
sns.scatterplot(x = 'net_bill', y = 'residues', data = tips)
plt.axhline(y=0, color='r', linestyle='--')

##### 4. Teste transformar ```net_bill``` no log e um polinômio. Escolha o melhor modelo.


In [145]:
# 4. Teste transformar net_bill no log e um polinômio. Escolha o melhor modelo.
y, X = patsy.dmatrices('np.log(tip) ~ np.log(net_bill)', tips)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

In [146]:
y, X = patsy.dmatrices('np.log(tip) ~ np.power(net_bill, 2)', tips)
modelo = sm.OLS(y, X).fit() 
print(modelo.summary())

##### 5. Do modelo final deste item, calcule o $R^2$ na escala de ```tip``` (sem o log). Compare com o modelo do item 1. Qual tem melhor coeficiente de determinação?

In [147]:
# Calcular o R^2 na escala de tip (sem o log) para o modelo do item 4
predicted_tips_log_net_bill = modelo.predict(X)  # Prever os valores de tip usando o modelo
predicted_tips = np.exp(predicted_tips_log_net_bill)  # Desfazer o log dos valores de tip
residuals = tips['tip'] - predicted_tips  # Calcular os resíduos

# Calcular R^2
SST = np.sum((tips['tip'] - np.mean(tips['tip'])) ** 2)
SSR = np.sum(residuals ** 2)
R2 = 1 - (SSR / SST)

print("R^2 na escala de 'tip' (sem o log) para o modelo do item 4:", R2)

# Agora, vamos comparar com o R^2 do modelo do item 1
print("R^2 do modelo do item 1:", modelo1.rsquared)

In [148]:
# O melhor modelo é o que tem a variável net_bill sem transformação. mas o modelo com a variável net_bill em log é melhor que o modelo com a variável net_bill ao quadrado.

### III. Previsão de renda

Vamos trabalhar a base que você vai usar no projeto do final deste ciclo.

Carregue a base ```previsao_de_renda.csv```.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Ajuste um modelo de regressão linear simples para explicar ```renda``` como variável resposta, por ```tempo_emprego``` como variável explicativa. Observe que há muitas observações nessa tabela. Utilize os recursos que achar necessário.
2. Faça uma análise de resíduos. Com os recursos vistos neste módulo, como você melhoraria esta regressão?
3. Ajuste um modelo de regressão linear múltipla para explicar ```renda``` (ou uma transformação de ```renda```) de acordo com as demais variáveis.
4. Remova as variáveis não significantes e ajuste novamente o modelo. Interprete os parâmetros
5. Faça uma análise de resíduos. Avalie a qualidade do ajuste.

In [149]:
previsao_de_renda = pd.read_csv('../../../../Datasets/previsao_de_renda.csv', sep = ',')
previsao_de_renda.head()

In [150]:
previsao_de_renda.info()

In [151]:
# tratando nan values
previsao_de_renda.isna().sum()

In [152]:
previsao_de_renda.dropna(inplace = True)
previsao_de_renda['tempo_emprego'].isna().sum()

In [153]:

# Calcular as correlações apenas com a variável "renda"
correlacoes_renda = previsao_de_renda.corr()['renda'].drop('renda')

# Ordenar as correlações por valor absoluto em ordem decrescente
correlacoes_renda_sorted = correlacoes_renda.abs().sort_values(ascending=False)

# Selecionar apenas as correlações ordenadas
correlacoes_renda_sorted = correlacoes_renda[correlacoes_renda_sorted.index]

# Criar o heatmap com as correlações ordenadas
sns.heatmap(correlacoes_renda_sorted.to_frame(), annot=True, cmap='coolwarm', fmt=".2f")



##### 1. Ajuste um modelo de regressão linear simples para explicar ```renda``` como variável resposta, por ```tempo_emprego``` como variável explicativa. Observe que há muitas observações nessa tabela. Utilize os recursos que achar necessário.

In [154]:

y, X = patsy.dmatrices('renda ~ tempo_emprego', previsao_de_renda)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

##### 2. Faça uma análise de resíduos. Com os recursos vistos neste módulo, como você melhoraria esta regressão?


In [155]:

previsao_de_renda['residues'] = modelo.resid
sns.scatterplot(x = 'tempo_emprego', y = 'residues', data = previsao_de_renda)  
plt.axhline(y=0, color='r', linestyle='--')

In [156]:
previsao_de_renda['predicted'] = modelo.fittedvalues
sns.scatterplot(x = 'predicted', y = 'renda', data = previsao_de_renda)
plt.plot(previsao_de_renda['predicted'], previsao_de_renda['predicted'], color = 'red')

A regressão não é adequada. Para melhorar a regressão simples, poderíamos tentar transformar a variável tempo_emprego ou utilizar outra variável explicativa.

##### 3. Ajuste um modelo de regressão linear múltipla para explicar ```renda``` (ou uma transformação de ```renda```) de acordo com as demais variáveis.

In [ ]:


numerical_columns = previsao_de_renda.select_dtypes(include = ['float64', 'int64'])
numerical_columns = numerical_columns.drop(columns = ['index', 'renda', 'residues'])
numerical_columns = numerical_columns.columns.tolist()
# Criando a fórmula para a matriz de design
formula = 'renda ~ ' + ' + '.join(numerical_columns)
y, X = patsy.dmatrices(formula, previsao_de_renda)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

In [ ]:

y, X = patsy.dmatrices('renda ~ tempo_emprego + idade + qt_pessoas_residencia', previsao_de_renda)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

##### 4. Remova as variáveis não significantes e ajuste novamente o modelo. Interprete os parâmetros

In [ ]:
import itertools

# Variáveis para as transformações
transformacoes = ['none', 'log', 'power']

# Lista de variáveis independentes
variaveis_independentes = ['tempo_emprego', 'idade', 'qt_pessoas_residencia']

# Gerar todas as combinações possíveis das transformações
combinacoes = list(itertools.product(transformacoes, repeat=len(variaveis_independentes)))

# Lista para armazenar as fórmulas
formulas = []
for i in ['np.log(renda)', 'renda']:
    formula_aux = i + ' ~ '
# Criar as fórmulas para cada combinação
    for combo in combinacoes:
        formula = formula_aux
        for i, var in enumerate(variaveis_independentes):
            if combo[i] == 'log':
                formula += f'np.log({var})'
            elif combo[i] == 'power':
                formula += f'np.power({var}, 2)'
            else:
                formula += var
            if i < len(variaveis_independentes) - 1:
                formula += ' + '
        formulas.append(formula)
for i in formulas:
    print(i)

In [ ]:
# testando várias combinações de transformações logarítmicas e polinomiais

# criando lista de fórmulas para testar várias combinações semelhante a um grid search


best_r2 = 0

for i in formulas:
    y, X = patsy.dmatrices(i, previsao_de_renda)
    modelo = sm.OLS(y, X).fit()
    if modelo.rsquared > best_r2:
        best_r2 = modelo.rsquared
        best_formula = i
    # imprimindo apenas o R2
    print(f'R2: {modelo.rsquared}')    
    print(f'Formula: {i}')

print(f'Melhor R2: {best_r2}')
print(f'Melhor formula: {best_formula}')

In [72]:
y, X = patsy.dmatrices('np.log(renda) ~ np.power(tempo_emprego, 2) + np.power(idade, 2) + np.log(qt_pessoas_residencia) + sexo + tipo_renda + posse_de_imovel', previsao_de_renda)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

##### 5. Faça uma análise de resíduos. Avalie a qualidade do ajuste.

In [73]:
previsao_de_renda['residues'] = modelo.resid
sns.scatterplot(x = 'renda', y = 'residues', data = previsao_de_renda)
plt.axhline(y=0, color='r', linestyle='--')

In [74]:
previsao_de_renda['predicted'] = modelo.fittedvalues
sns.scatterplot(x = 'predicted', y = 'renda', data = previsao_de_renda)
plt.plot(previsao_de_renda['predicted'], previsao_de_renda['predicted'], color = 'red')